## DESCARGANDO LOS MODELOS PREENTRENADOS DE FACEBOOK

In [1]:
import fasttext
import pandas as pd
import numpy as np

In [ ]:
from gensim.models import KeyedVectors

modelo_ing = KeyedVectors.load_word2vec_format("../vectorizadores/wiki-news-300d-1M.vec", binary=False)

### Cargando dataset en ingles

In [3]:
inglesdt = pd.read_csv('../data_process/dataInglesTr.csv')

In [7]:
inglesdt.drop(['score'], axis=1, inplace=True)


In [8]:
inglesdt.head()

,processed_text,sentiment,language
0,bad talented illustrations context degrade sit...,enojado,en
1,tape measure level work good extend full 25 fe...,enojado,en
2,place large order seller include plant ship fa...,insatisfecho,en
3,no idea thing work n't help no specifications ...,neutral,en
4,product arrive canister full someone else 's d...,enojado,en


In [4]:
def obtener_vector_promedio(texto, modelo_fasttext):
    palabras = texto.split()
    vectores = [modelo_fasttext[word] for word in palabras if word in modelo_fasttext]  # Cambio aquí

    if not vectores:  # Si no hay palabras en el texto
        return np.zeros(300)  # Vector de ceros de 300 dimensiones

    return np.mean(vectores, axis=0)  # Promedio de los vectores


In [9]:
# Aplicar a la columna de reseñas en inglés
inglesdt['embedding'] = inglesdt['processed_text'].apply(lambda x: obtener_vector_promedio(x, modelo_ing))

In [10]:
inglesdt.head()

,processed_text,sentiment,language,embedding
0,bad talented illustrations context degrade sit...,enojado,en,"[-0.03916667, -0.066183336, -0.051483333, 0.00..."
1,tape measure level work good extend full 25 fe...,enojado,en,"[-0.03249623, -0.018601885, -0.038154714, 0.01..."
2,place large order seller include plant ship fa...,insatisfecho,en,"[0.015111112, -0.021373738, -0.036773734, -0.0..."
3,no idea thing work n't help no specifications ...,neutral,en,"[-0.027861763, -0.028435295, -0.054567646, 0.0..."
4,product arrive canister full someone else 's d...,enojado,en,"[-0.034793336, -0.016376667, -0.0040333355, 0...."


In [12]:
inglesdt.to_csv('../data_process/vectores/vectoresEnglish.csv', index=False)

### SPANISH DATA AB PROCESS

In [18]:
# Cargar el modelo preentrenado en inglés
modelo_inges = fasttext.load_model("../vectorizadores/cc.es.300.bin")

In [16]:
def obtener_vector_promedioEs(texto, modelo_fasttext):
    palabras = texto.split()
    vectores = [modelo_fasttext.get_word_vector(palabra) for palabra in palabras]

    if not vectores:  # Si no hay palabras en el texto
        return np.zeros(300)  # Vector de ceros de 300 dimensiones

    return np.mean(vectores, axis=0)  # Promedio de los vectores


In [14]:
SpanishData = pd.read_csv('../data_process/SpanishData.csv')

In [15]:
SpanishData.head()

,stars,language,processed_text,sentiment
0,3,es,máquina mejor poco ruido cortando perfecta muy...,contento
1,3,es,meses funcionó muy protege teléfono golpes fun...,insatisfecho
2,3,es,caja bonito,contento
3,3,es,esperaba cosa ruidoso poco aire pequeña brisa ...,insatisfecho
4,3,es,medidas no correctas sofá mide supone funda pl...,insatisfecho


In [19]:
# Aplicar la función a la columna 'review' en inglés
SpanishData['embedding'] = SpanishData['processed_text'].apply(lambda x: obtener_vector_promedio(x, modelo_inges))

In [20]:
SpanishData.head()

,stars,language,processed_text,sentiment,embedding
0,3,es,máquina mejor poco ruido cortando perfecta muy...,contento,"[-0.018338734, -0.032938536, 0.024386868, 0.01..."
1,3,es,meses funcionó muy protege teléfono golpes fun...,insatisfecho,"[0.005521389, -0.018885465, 0.014168648, -0.01..."
2,3,es,caja bonito,contento,"[-0.036779333, -0.014089687, 0.018141113, -0.1..."
3,3,es,esperaba cosa ruidoso poco aire pequeña brisa ...,insatisfecho,"[0.0013987708, -0.017774584, -0.021600656, -0...."
4,3,es,medidas no correctas sofá mide supone funda pl...,insatisfecho,"[0.0049701906, -0.023959292, -0.021284338, 0.0..."


In [21]:
SpanishData.to_csv('../data_process/vectores/vectoreSpanish.csv', index=False)

In [29]:
SpanishData.drop(['stars'], axis=1, inplace=True)

In [30]:
SpanishData.head()

,language,sentiment,embedding
0,es,contento,"[-0.018338734, -0.032938536, 0.024386868, 0.01..."
1,es,insatisfecho,"[0.005521389, -0.018885465, 0.014168648, -0.01..."
2,es,contento,"[-0.036779333, -0.014089687, 0.018141113, -0.1..."
3,es,insatisfecho,"[0.0013987708, -0.017774584, -0.021600656, -0...."
4,es,insatisfecho,"[0.0049701906, -0.023959292, -0.021284338, 0.0..."


In [ ]:
inglesdt.drop(['processed_text'], axis=1, inplace=True)

KeyError: "['stars'] not found in axis"

In [27]:
inglesdt.head()


,sentiment,language,embedding
0,enojado,en,"[-0.03916667, -0.066183336, -0.051483333, 0.00..."
1,enojado,en,"[-0.03249623, -0.018601885, -0.038154714, 0.01..."
2,insatisfecho,en,"[0.015111112, -0.021373738, -0.036773734, -0.0..."
3,neutral,en,"[-0.027861763, -0.028435295, -0.054567646, 0.0..."
4,enojado,en,"[-0.034793336, -0.016376667, -0.0040333355, 0...."


## UNIENDO LOS EMBEDDINGS

In [31]:
df_vector_merge = pd.concat([inglesdt, SpanishData], axis=0, ignore_index=True)

In [32]:
if isinstance(df_vector_merge['embedding'].iloc[0], str):
    df_vector_merge['embedding'] = df_vector_merge['embedding'].apply(lambda x: np.array(eval(x)))

In [35]:
df_vector_merge['embedding'] = df_vector_merge['embedding'].apply(
    lambda x: x.astype(np.float32) if isinstance(x, np.ndarray) else np.zeros(300, dtype=np.float32))

In [37]:
df_vector_merge.to_parquet('../data_process/vectores/EmbeddingsFinal.parquet', engine='pyarrow')

In [33]:
df_vector_merge.head()

,sentiment,language,embedding
0,enojado,en,"[-0.03916667, -0.066183336, -0.051483333, 0.00..."
1,enojado,en,"[-0.03249623, -0.018601885, -0.038154714, 0.01..."
2,insatisfecho,en,"[0.015111112, -0.021373738, -0.036773734, -0.0..."
3,neutral,en,"[-0.027861763, -0.028435295, -0.054567646, 0.0..."
4,enojado,en,"[-0.034793336, -0.016376667, -0.0040333355, 0...."
